# Content Based Recommender Systems

In this notebook, we will explore Content-based recommendation systems and implement a simple version of one using Python and the Pandas library.

In [1]:
import numpy as np
import pandas as pd

## Load the Dataset

In [135]:
movies_df = pd.read_csv('movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Preprocessing

Let's remove the year from the title column and store in a new year column.

In [3]:
movies_df['year'] = movies_df.title.str.extract('(\d\d\d\d)',expand=False)

In [4]:
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

In [5]:
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

In [6]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also split the values in the Genres column into a list of Genres to simplify future use.

In [7]:
movies_df['genres'] = movies_df.genres.str.split('|')

In [8]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


We will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature.

In [9]:
moviesWithGenres_df = movies_df.copy()

In [10]:
for index,row in movies_df.iterrows():
    for genres in row['genres']:
        moviesWithGenres_df.at[index,genres] = 1

In [11]:
moviesWithGenres_df = moviesWithGenres_df.fillna(0)

In [12]:
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Content-Based Filtering

In [13]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 

In [14]:
InputMovies = pd.DataFrame(userInput)
InputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


Add **movieId** to input user

In [15]:
InputId = movies_df[movies_df['title'].isin(InputMovies['title'].tolist())]
InputMovies = pd.merge(InputId, InputMovies)
InputMovies

,movieId,title,genres,year,rating
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,3.5
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,2.0
2,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,5.0
3,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,4.5
4,1968,"Breakfast Club, The","[Comedy, Drama]",1985,5.0


In [16]:
InputMovies = InputMovies.drop(['genres','year'],axis=1)
InputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.

In [78]:
usermovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(InputMovies['movieId'])]
usermovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
usermovies = usermovies.iloc[:,4:]
usermovies = usermovies.reset_index(drop=True)
usermovies

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
userProfile = usermovies.transpose().dot(InputMovies['rating'])
#The user profile
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [60]:
usermovies.iloc[[1]]

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
len(InputMovies[['rating']])

5

In [70]:
InputMovies[['rating']].loc[0].values

array([3.5])

In [83]:
InputMovies[['rating']]

,rating
0,3.5
1,2.0
2,5.0
3,4.5
4,5.0


In [81]:
for i in range(len(InputMovies[['rating']])):
     usermovies.iloc[i] = usermovies.iloc[i] * InputMovies[['rating']].loc[i].values

In [82]:
usermovies

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,3.5,3.5,3.5,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.5,4.5,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
d= usermovies.sum(axis=0)

In [117]:
d

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [118]:
a = d.sum()

In [119]:
a.sum()

71.5

In [120]:
u = d/71.5

In [121]:
u

Adventure             0.139860
Animation             0.111888
Children              0.076923
Comedy                0.188811
Fantasy               0.076923
Romance               0.000000
Drama                 0.139860
Action                0.062937
Crime                 0.069930
Thriller              0.069930
Horror                0.000000
Mystery               0.000000
Sci-Fi                0.062937
IMAX                  0.000000
Documentary           0.000000
War                   0.000000
Musical               0.000000
Western               0.000000
Film-Noir             0.000000
(no genres listed)    0.000000
dtype: float64

In [122]:
u.sort_values(ascending=False)

Comedy                0.188811
Adventure             0.139860
Drama                 0.139860
Animation             0.111888
Children              0.076923
Fantasy               0.076923
Crime                 0.069930
Thriller              0.069930
Sci-Fi                0.062937
Action                0.062937
Mystery               0.000000
Horror                0.000000
Film-Noir             0.000000
IMAX                  0.000000
Romance               0.000000
Documentary           0.000000
War                   0.000000
Musical               0.000000
Western               0.000000
(no genres listed)    0.000000
dtype: float64

In [123]:
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## candidate movie

In [288]:
genretable = moviesWithGenres_df.set_index('movieId')

In [289]:
genretable.head()

,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [290]:
genretable = genretable.iloc[:,3:]
genretable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
151703,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [291]:
u

Adventure             0.139860
Animation             0.111888
Children              0.076923
Comedy                0.188811
Fantasy               0.076923
Romance               0.000000
Drama                 0.139860
Action                0.062937
Crime                 0.069930
Thriller              0.069930
Horror                0.000000
Mystery               0.000000
Sci-Fi                0.062937
IMAX                  0.000000
Documentary           0.000000
War                   0.000000
Musical               0.000000
Western               0.000000
Film-Noir             0.000000
(no genres listed)    0.000000
dtype: float64

In [292]:
d.sum()

71.5

In [293]:
genretable['Animation'] * 2

movieId
1         2.0
2         0.0
3         0.0
4         0.0
5         0.0
         ... 
151697    0.0
151701    0.0
151703    0.0
151709    0.0
151711    0.0
Name: Animation, Length: 34208, dtype: float64

In [294]:
genretable[[genretable.columns[0]]] *2

,Adventure
movieId,
1,2.0
2,2.0
3,0.0
4,0.0
5,0.0
...,...
151697,0.0
151701,0.0
151703,0.0


In [295]:
range(len(u))

range(0, 20)

In [296]:
len(genretable.columns)

20

In [299]:
u

Adventure             0.139860
Animation             0.111888
Children              0.076923
Comedy                0.188811
Fantasy               0.076923
Romance               0.000000
Drama                 0.139860
Action                0.062937
Crime                 0.069930
Thriller              0.069930
Horror                0.000000
Mystery               0.000000
Sci-Fi                0.062937
IMAX                  0.000000
Documentary           0.000000
War                   0.000000
Musical               0.000000
Western               0.000000
Film-Noir             0.000000
(no genres listed)    0.000000
dtype: float64

In [297]:
for i in range(len(u)):
    genretable[genretable.columns[i]] = genretable[genretable.columns[i]] * u[i]

In [298]:
genretable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,0.13986,0.111888,0.076923,0.188811,0.076923,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.13986,0.000000,0.076923,0.000000,0.076923,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.000000,0.000000,0.188811,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.000000,0.000000,0.188811,0.000000,0.0,0.13986,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.00000,0.000000,0.000000,0.188811,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151697,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.06993,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151701,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151703,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.13986,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [300]:
recoom = genretable.sum(axis=1)

In [301]:
recoom

movieId
1         0.594406
2         0.293706
3         0.188811
4         0.328671
5         0.188811
            ...   
151697    0.069930
151701    0.000000
151703    0.139860
151709    0.202797
151711    0.000000
Length: 34208, dtype: float64

In [303]:
recoom = recoom.sort_values(ascending=False)
recoom.head()

movieId
5018      0.748252
26093     0.734266
27344     0.720280
148775    0.685315
117646    0.678322
dtype: float64

## Now here's the recommendation table!

In [311]:
movies_df.loc[movies_df['movieId'].isin(recoom.head(20).keys())]

,movieId,title,genres
664,673,Space Jam (1996),Adventure|Animation|Children|Comedy|Fantasy|Sc...
1824,1907,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...
2902,2987,Who Framed Roger Rabbit? (1988),Adventure|Animation|Children|Comedy|Crime|Fant...
4923,5018,Motorama (1991),Adventure|Comedy|Crime|Drama|Fantasy|Mystery|S...
6793,6902,Interstate 60 (2002),Adventure|Comedy|Drama|Fantasy|Mystery|Sci-Fi|...
8605,26093,"Wonderful World of the Brothers Grimm, The (1962)",Adventure|Animation|Children|Comedy|Drama|Fant...
8783,26340,"Twelve Tasks of Asterix, The (Les douze travau...",Action|Adventure|Animation|Children|Comedy|Fan...
9296,27344,Revolutionary Girl Utena: Adolescence of Utena...,Action|Adventure|Animation|Comedy|Drama|Fantas...
9825,32031,Robots (2005),Adventure|Animation|Children|Comedy|Fantasy|Sc...
11716,51632,Atlantis: Milo's Return (2003),Action|Adventure|Animation|Children|Comedy|Fan...
